# Unity ML-Agents: Обучение через внешний API с DQN и PPO

Этот notebook демонстрирует обучение ML-агента через внешний API с Unity, используя exe-файл обучающей среды.

## Содержание:
1. [Установка и настройка окружения](#1)
2. [Подключение к внешней Unity среде](#2)
3. [Реализация DQN алгоритма](#3)
4. [Реализация PPO алгоритма](#4)
5. [Сравнение результатов](#5)
6. [Troubleshooting](#6)

## 1. Установка и настройка окружения <a id="1"></a>

Устанавливаем необходимые библиотеки для работы с Unity ML-Agents через внешний API.

In [ ]:
# Установка основных зависимостей
!pip install mlagents>=1.0.0
!pip install gym-unity>=0.29.0
!pip install torch>=2.0.0
!pip install torchvision>=0.15.0
!pip install stable-baselines3>=2.0.0
!pip install tensorboard>=2.13.0
!pip install matplotlib>=3.7.0
!pip install numpy>=1.24.0
!pip install pandas>=2.0.0
!pip install seaborn>=0.12.0
!pip install tqdm>=4.65.0
!pip install pillow>=9.5.0

In [ ]:
# Импорт всех необходимых библиотек
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque, namedtuple
import random
import os
import time
import pickle
from tqdm import tqdm
import pandas as pd

# Unity ML-Agents imports
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.side_channel.environment_parameters_channel import EnvironmentParametersChannel

# Stable Baselines3 imports for PPO
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.monitor import Monitor

# Настройка для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Настройка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")

# Настройка стиля графиков
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Все библиотеки успешно импортированы!")

## 2. Подключение к внешней Unity среде <a id="2"></a>

Настраиваем подключение к Unity exe-файлу через API.

In [ ]:
# Настройка параметров подключения
class UnityConfig:
    def __init__(self):
        # Путь к Unity exe-файлу (измените на свой путь)
        self.unity_exe_path = "./YourUnityEnvironment.exe"  # Замените на ваш путь
        
        # Настройки подключения
        self.base_port = 5005
        self.no_graphics = False  # Установите True для headless режима
        self.time_scale = 20.0    # Ускорение обучения
        
        # Параметры среды
        self.max_steps = 1000000  # Максимальное количество шагов
        self.num_envs = 1         # Количество параллельных сред
        
        # Параметры обучения
        self.episode_length = 1000
        self.target_reward = 10.0

config = UnityConfig()
print(f"Конфигурация Unity среды:")
print(f"  - Путь к exe: {config.unity_exe_path}")
print(f"  - Порт: {config.base_port}")
print(f"  - Без графики: {config.no_graphics}")
print(f"  - Скорость времени: {config.time_scale}")

In [ ]:
def create_unity_environment(config):
    """
    Создает подключение к Unity среде.
    """
    # Создаем каналы для настройки среды
    engine_config_channel = EngineConfigurationChannel()
    env_params_channel = EnvironmentParametersChannel()
    
    # Создаем среду
    try:
        env = UnityEnvironment(
            file_name=config.unity_exe_path,
            base_port=config.base_port,
            no_graphics=config.no_graphics,
            side_channels=[engine_config_channel, env_params_channel]
        )
        
        # Настраиваем параметры среды
        engine_config_channel.set_configuration_parameters(
            time_scale=config.time_scale,
            target_frame_rate=60,
            capture_frame_rate=60
        )
        
        print("✅ Unity среда успешно создана!")
        return env, engine_config_channel, env_params_channel
        
    except Exception as e:
        print(f"❌ Ошибка при создании Unity среды: {e}")
        print("\n🔧 Проверьте:")
        print("  1. Путь к exe-файлу корректен")
        print("  2. Exe-файл запускается")
        print("  3. Порт не занят другим процессом")
        print("  4. Unity среда собрана с ML-Agents")
        return None, None, None

# Создаем среду (раскомментируйте когда у вас будет exe-файл)
# env, engine_channel, params_channel = create_unity_environment(config)

# Для демонстрации создаем mock-среду
print("⚠️  Для демонстрации используется mock-среда")
print("   Замените на реальную Unity среду когда она будет готова")

In [ ]:
def get_environment_info(env):
    """
    Получает информацию о Unity среде.
    """
    # Сброс среды для получения информации
    env.reset()
    
    # Получаем имена behavior
    behavior_names = list(env.behavior_specs.keys())
    print(f"Найдено {len(behavior_names)} behavior(s): {behavior_names}")
    
    # Анализируем каждый behavior
    for behavior_name in behavior_names:
        spec = env.behavior_specs[behavior_name]
        print(f"\n📊 Behavior: {behavior_name}")
        print(f"  - Размер наблюдений: {spec.observation_specs[0].shape}")
        print(f"  - Количество действий: {spec.action_spec.continuous_size + spec.action_spec.discrete_size}")
        print(f"  - Непрерывные действия: {spec.action_spec.continuous_size}")
        print(f"  - Дискретные действия: {spec.action_spec.discrete_size}")
        
        if spec.action_spec.discrete_size > 0:
            print(f"  - Ветви дискретных действий: {spec.action_spec.discrete_branches}")
    
    return behavior_names

# Получаем информацию о среде (раскомментируйте для реальной среды)
# behavior_names = get_environment_info(env)

# Для демонстрации создаем mock-параметры
class MockEnvironmentInfo:
    def __init__(self):
        self.behavior_names = ["MyAgent"]
        self.observation_size = 8
        self.action_size = 4
        self.continuous_actions = 2
        self.discrete_actions = 2

mock_env_info = MockEnvironmentInfo()
print("🎯 Mock-среда создана для демонстрации:")
print(f"  - Размер наблюдений: {mock_env_info.observation_size}")
print(f"  - Размер действий: {mock_env_info.action_size}")
print(f"  - Непрерывные действия: {mock_env_info.continuous_actions}")
print(f"  - Дискретные действия: {mock_env_info.discrete_actions}")

## 3. Реализация DQN алгоритма <a id="3"></a>

Реализуем Deep Q-Network с полностью настраиваемой архитектурой.

In [ ]:
class ConfigurableQNetwork(nn.Module):
    """
    Полностью настраиваемая Q-сеть для DQN.
    Вы можете изменить архитектуру, добавить слои, изменить активации.
    """
    
    def __init__(self, state_size, action_size, hidden_layers=None, activation='relu', dropout=0.0):
        super(ConfigurableQNetwork, self).__init__()
        
        # Параметры по умолчанию
        if hidden_layers is None:
            hidden_layers = [256, 256, 128]  # Можете изменить архитектуру здесь
        
        self.state_size = state_size
        self.action_size = action_size
        self.hidden_layers = hidden_layers
        self.dropout = dropout
        
        # Выбор функции активации
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'leaky_relu':
            self.activation = nn.LeakyReLU()
        elif activation == 'elu':
            self.activation = nn.ELU()
        else:
            self.activation = nn.ReLU()
        
        # Создание слоев
        layers = []
        input_size = state_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(self.activation)
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            input_size = hidden_size
        
        # Выходной слой
        layers.append(nn.Linear(input_size, action_size))
        
        self.network = nn.Sequential(*layers)
        
        # Инициализация весов
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Инициализация весов Xavier/He."""
        if isinstance(module, nn.Linear):
            # Используем Xavier инициализацию для ReLU
            nn.init.xavier_uniform_(module.weight)
            nn.init.constant_(module.bias, 0)
    
    def forward(self, state):
        """Прямой проход."""
        return self.network(state)
    
    def get_action(self, state, epsilon=0.0):
        """Получение действия с epsilon-greedy стратегией."""
        if random.random() < epsilon:
            return random.randrange(self.action_size)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            q_values = self.forward(state_tensor)
            return q_values.argmax().item()

# Пример создания различных архитектур
print("🧠 Примеры различных архитектур Q-сети:")
print("\n1. Простая архитектура:")
simple_net = ConfigurableQNetwork(8, 4, hidden_layers=[64, 64])
print(f"   Параметры: {sum(p.numel() for p in simple_net.parameters())}")

print("\n2. Глубокая архитектура:")
deep_net = ConfigurableQNetwork(8, 4, hidden_layers=[512, 256, 128, 64])
print(f"   Параметры: {sum(p.numel() for p in deep_net.parameters())}")

print("\n3. Широкая архитектура:")
wide_net = ConfigurableQNetwork(8, 4, hidden_layers=[1024, 512])
print(f"   Параметры: {sum(p.numel() for p in wide_net.parameters())}")

print("\n4. Архитектура с dropout:")
dropout_net = ConfigurableQNetwork(8, 4, hidden_layers=[256, 256], dropout=0.3)
print(f"   Параметры: {sum(p.numel() for p in dropout_net.parameters())}")

print("\n✅ Все архитектуры созданы успешно!")

In [ ]:
class ReplayBuffer:
    """
    Буфер опыта для DQN.
    """
    
    def __init__(self, capacity=100000):
        self.buffer = deque(maxlen=capacity)
        self.experience = namedtuple('Experience', 
                                   ['state', 'action', 'reward', 'next_state', 'done'])
    
    def push(self, state, action, reward, next_state, done):
        """Добавляет опыт в буфер."""
        exp = self.experience(state, action, reward, next_state, done)
        self.buffer.append(exp)
    
    def sample(self, batch_size):
        """Выбирает случайную партию из буфера."""
        experiences = random.sample(self.buffer, batch_size)
        
        states = torch.FloatTensor([e.state for e in experiences]).to(device)
        actions = torch.LongTensor([e.action for e in experiences]).to(device)
        rewards = torch.FloatTensor([e.reward for e in experiences]).to(device)
        next_states = torch.FloatTensor([e.next_state for e in experiences]).to(device)
        dones = torch.BoolTensor([e.done for e in experiences]).to(device)
        
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        return len(self.buffer)

# Тестирование буфера
buffer = ReplayBuffer(10000)
print(f"💾 Буфер опыта создан, максимальный размер: {buffer.buffer.maxlen}")

# Добавляем тестовый опыт
for i in range(5):
    state = np.random.random(8)
    action = np.random.randint(0, 4)
    reward = np.random.random()
    next_state = np.random.random(8)
    done = np.random.choice([True, False])
    buffer.push(state, action, reward, next_state, done)

print(f"Добавлено {len(buffer)} опытов в буфер")

In [ ]:
class DQNAgent:
    """
    Агент DQN с полностью настраиваемыми параметрами.
    """
    
    def __init__(self, state_size, action_size, config=None):
        self.state_size = state_size
        self.action_size = action_size
        
        # Конфигурация по умолчанию
        if config is None:
            config = {
                'learning_rate': 0.001,
                'gamma': 0.99,
                'epsilon_start': 1.0,
                'epsilon_end': 0.01,
                'epsilon_decay': 0.995,
                'batch_size': 64,
                'buffer_size': 100000,
                'target_update_freq': 1000,
                'hidden_layers': [256, 256],
                'activation': 'relu',
                'dropout': 0.0,
                'optimizer': 'adam',
                'loss_function': 'huber'  # или 'mse'
            }
        
        self.config = config
        self.epsilon = config['epsilon_start']
        
        # Создаем сети
        self.q_network = ConfigurableQNetwork(
            state_size, action_size, 
            hidden_layers=config['hidden_layers'],
            activation=config['activation'],
            dropout=config['dropout']
        ).to(device)
        
        self.target_network = ConfigurableQNetwork(
            state_size, action_size,
            hidden_layers=config['hidden_layers'],
            activation=config['activation'],
            dropout=config['dropout']
        ).to(device)
        
        # Инициализируем target network
        self.target_network.load_state_dict(self.q_network.state_dict())
        
        # Оптимизатор
        if config['optimizer'] == 'adam':
            self.optimizer = optim.Adam(self.q_network.parameters(), lr=config['learning_rate'])
        elif config['optimizer'] == 'rmsprop':
            self.optimizer = optim.RMSprop(self.q_network.parameters(), lr=config['learning_rate'])
        elif config['optimizer'] == 'sgd':
            self.optimizer = optim.SGD(self.q_network.parameters(), lr=config['learning_rate'])
        
        # Буфер опыта
        self.replay_buffer = ReplayBuffer(config['buffer_size'])
        
        # Счетчики
        self.step_count = 0
        self.episode_count = 0
        
        # Метрики
        self.losses = []
        self.rewards = []
        self.epsilons = []
        
        print(f"🤖 DQN агент создан:")
        print(f"  - Архитектура: {config['hidden_layers']}")
        print(f"  - Активация: {config['activation']}")
        print(f"  - Оптимизатор: {config['optimizer']}")
        print(f"  - Learning rate: {config['learning_rate']}")
        print(f"  - Параметры сети: {sum(p.numel() for p in self.q_network.parameters())}")
    
    def get_action(self, state):
        """Получение действия с epsilon-greedy стратегией."""
        return self.q_network.get_action(state, self.epsilon)
    
    def step(self, state, action, reward, next_state, done):
        """Один шаг обучения."""
        # Добавляем в буфер
        self.replay_buffer.push(state, action, reward, next_state, done)
        
        self.step_count += 1
        
        # Обучение
        if len(self.replay_buffer) >= self.config['batch_size']:
            self.learn()
        
        # Обновление target network
        if self.step_count % self.config['target_update_freq'] == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())
    
    def learn(self):
        """Обучение на партии из буфера."""
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.config['batch_size'])
        
        # Текущие Q-значения
        current_q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Следующие Q-значения
        with torch.no_grad():
            next_q_values = self.target_network(next_states).max(1)[0]
            target_q_values = rewards + (self.config['gamma'] * next_q_values * ~dones)
        
        # Потеря
        if self.config['loss_function'] == 'huber':
            loss = F.smooth_l1_loss(current_q_values, target_q_values)
        else:
            loss = F.mse_loss(current_q_values, target_q_values)
        
        # Обновление
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        self.losses.append(loss.item())
        
        # Обновление epsilon
        self.epsilon = max(self.config['epsilon_end'], 
                          self.epsilon * self.config['epsilon_decay'])
        self.epsilons.append(self.epsilon)
    
    def save_model(self, filepath):
        """Сохранение модели."""
        torch.save({
            'q_network_state_dict': self.q_network.state_dict(),
            'target_network_state_dict': self.target_network.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'config': self.config,
            'step_count': self.step_count,
            'episode_count': self.episode_count,
            'epsilon': self.epsilon
        }, filepath)
        print(f"💾 Модель сохранена: {filepath}")
    
    def load_model(self, filepath):
        """Загрузка модели."""
        checkpoint = torch.load(filepath, map_location=device)
        self.q_network.load_state_dict(checkpoint['q_network_state_dict'])
        self.target_network.load_state_dict(checkpoint['target_network_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.step_count = checkpoint['step_count']
        self.episode_count = checkpoint['episode_count']
        self.epsilon = checkpoint['epsilon']
        print(f"📂 Модель загружена: {filepath}")

# Создаем DQN агента
dqn_config = {
    'learning_rate': 0.001,
    'gamma': 0.99,
    'epsilon_start': 1.0,
    'epsilon_end': 0.01,
    'epsilon_decay': 0.995,
    'batch_size': 64,
    'buffer_size': 100000,
    'target_update_freq': 1000,
    'hidden_layers': [256, 256, 128],  # Измените архитектуру здесь
    'activation': 'relu',  # relu, tanh, leaky_relu, elu
    'dropout': 0.0,
    'optimizer': 'adam',  # adam, rmsprop, sgd
    'loss_function': 'huber'  # huber, mse
}

dqn_agent = DQNAgent(mock_env_info.observation_size, mock_env_info.action_size, dqn_config)
print("\n✅ DQN агент готов к обучению!")

### Обучение DQN агента

In [ ]:
def train_dqn_agent(agent, env=None, episodes=1000, max_steps=1000, save_freq=100):
    """
    Обучение DQN агента.
    """
    print(f"🚀 Начинаем обучение DQN на {episodes} эпизодов...")
    
    episode_rewards = []
    episode_lengths = []
    
    # Прогресс бар
    pbar = tqdm(range(episodes), desc="DQN Training")
    
    for episode in pbar:
        # Для демонстрации используем mock-среду
        if env is None:
            # Mock environment для демонстрации
            state = np.random.random(agent.state_size)
            episode_reward = 0
            steps = 0
            
            for step in range(max_steps):
                # Получаем действие
                action = agent.get_action(state)
                
                # Mock переход
                next_state = np.random.random(agent.state_size)
                reward = np.random.random() - 0.5  # Случайная награда
                done = np.random.random() < 0.01  # 1% шанс завершения
                
                # Обучение агента
                agent.step(state, action, reward, next_state, done)
                
                state = next_state
                episode_reward += reward
                steps += 1
                
                if done:
                    break
        else:
            # Реальная Unity среда
            env.reset()
            decision_steps, terminal_steps = env.get_steps(agent.behavior_name)
            
            episode_reward = 0
            steps = 0
            
            while len(decision_steps) > 0 and steps < max_steps:
                # Получаем наблюдения
                observations = decision_steps.obs[0]
                
                # Получаем действия для всех агентов
                actions = []
                for obs in observations:
                    action = agent.get_action(obs)
                    actions.append(action)
                
                # Отправляем действия в Unity
                action_tuple = ActionTuple(discrete=np.array([actions]))
                env.set_actions(agent.behavior_name, action_tuple)
                env.step()
                
                # Получаем результаты
                decision_steps, terminal_steps = env.get_steps(agent.behavior_name)
                
                # Обработка terminal steps
                for i, agent_id in enumerate(terminal_steps.agent_id):
                    reward = terminal_steps.reward[i]
                    episode_reward += reward
                    # Добавить обучение агента здесь
                
                steps += 1
        
        episode_rewards.append(episode_reward)
        episode_lengths.append(steps)
        agent.episode_count += 1
        
        # Обновление прогресса
        if episode % 10 == 0:
            avg_reward = np.mean(episode_rewards[-10:])
            pbar.set_postfix({
                'Avg Reward': f'{avg_reward:.2f}',
                'Epsilon': f'{agent.epsilon:.3f}',
                'Buffer': len(agent.replay_buffer)
            })
        
        # Сохранение модели
        if episode % save_freq == 0 and episode > 0:
            agent.save_model(f'dqn_model_episode_{episode}.pth')
    
    return episode_rewards, episode_lengths

# Обучение DQN (демонстрация на mock-среде)
print("⚠️  Демонстрация обучения на mock-среде (замените на реальную Unity среду)")
dqn_rewards, dqn_lengths = train_dqn_agent(dqn_agent, episodes=100)

print(f"\n✅ Обучение DQN завершено!")
print(f"  - Средняя награда: {np.mean(dqn_rewards):.2f}")
print(f"  - Средняя длина эпизода: {np.mean(dqn_lengths):.2f}")
print(f"  - Финальный epsilon: {dqn_agent.epsilon:.3f}")

## 4. Реализация PPO алгоритма <a id="4"></a>

Реализуем Proximal Policy Optimization с настраиваемой архитектурой.

In [ ]:
class ConfigurablePPONetwork(nn.Module):
    """
    Настраиваемая сеть для PPO с policy и value головами.
    """
    
    def __init__(self, state_size, action_size, hidden_layers=None, activation='relu', 
                 dropout=0.0, separate_networks=False):
        super(ConfigurablePPONetwork, self).__init__()
        
        if hidden_layers is None:
            hidden_layers = [256, 256]
        
        self.state_size = state_size
        self.action_size = action_size
        self.separate_networks = separate_networks
        
        # Выбор активации
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'leaky_relu':
            self.activation = nn.LeakyReLU()
        elif activation == 'elu':
            self.activation = nn.ELU()
        else:
            self.activation = nn.ReLU()
        
        if separate_networks:
            # Отдельные сети для policy и value
            self.policy_net = self._build_network(state_size, hidden_layers, dropout)
            self.value_net = self._build_network(state_size, hidden_layers, dropout)
            
            self.policy_head = nn.Linear(hidden_layers[-1], action_size)
            self.value_head = nn.Linear(hidden_layers[-1], 1)
        else:
            # Общая сеть с разными головами
            self.shared_net = self._build_network(state_size, hidden_layers, dropout)
            
            self.policy_head = nn.Linear(hidden_layers[-1], action_size)
            self.value_head = nn.Linear(hidden_layers[-1], 1)
        
        # Инициализация весов
        self.apply(self._init_weights)
    
    def _build_network(self, input_size, hidden_layers, dropout):
        """Создание сети."""
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(self.activation)
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_size = hidden_size
        
        return nn.Sequential(*layers)
    
    def _init_weights(self, module):
        """Инициализация весов."""
        if isinstance(module, nn.Linear):
            nn.init.orthogonal_(module.weight, np.sqrt(2))
            nn.init.constant_(module.bias, 0)
    
    def forward(self, state):
        """Прямой проход."""
        if self.separate_networks:
            policy_features = self.policy_net(state)
            value_features = self.value_net(state)
            
            policy_logits = self.policy_head(policy_features)
            value = self.value_head(value_features)
        else:
            shared_features = self.shared_net(state)
            
            policy_logits = self.policy_head(shared_features)
            value = self.value_head(shared_features)
        
        return policy_logits, value
    
    def get_action_and_value(self, state):
        """Получение действия и значения."""
        policy_logits, value = self.forward(state)
        
        # Создаем распределение
        dist = torch.distributions.Categorical(logits=policy_logits)
        
        # Выбираем действие
        action = dist.sample()
        
        return action, dist.log_prob(action), value, dist.entropy()
    
    def evaluate_actions(self, state, actions):
        """Оценка действий для обучения."""
        policy_logits, value = self.forward(state)
        
        dist = torch.distributions.Categorical(logits=policy_logits)
        
        action_log_probs = dist.log_prob(actions)
        entropy = dist.entropy()
        
        return action_log_probs, value, entropy

# Пример создания различных PPO архитектур
print("🧠 Примеры различных архитектур PPO-сети:")

print("\n1. Общая сеть:")
shared_net = ConfigurablePPONetwork(8, 4, hidden_layers=[256, 256], separate_networks=False)
print(f"   Параметры: {sum(p.numel() for p in shared_net.parameters())}")

print("\n2. Отдельные сети:")
separate_net = ConfigurablePPONetwork(8, 4, hidden_layers=[256, 256], separate_networks=True)
print(f"   Параметры: {sum(p.numel() for p in separate_net.parameters())}")

print("\n3. Глубокая архитектура:")
deep_ppo_net = ConfigurablePPONetwork(8, 4, hidden_layers=[512, 256, 128], dropout=0.2)
print(f"   Параметры: {sum(p.numel() for p in deep_ppo_net.parameters())}")

print("\n✅ Все PPO архитектуры созданы успешно!")

In [ ]:
class PPOAgent:
    """
    PPO агент с полностью настраиваемыми параметрами.
    """
    
    def __init__(self, state_size, action_size, config=None):
        self.state_size = state_size
        self.action_size = action_size
        
        # Конфигурация по умолчанию
        if config is None:
            config = {
                'learning_rate': 3e-4,
                'gamma': 0.99,
                'gae_lambda': 0.95,
                'clip_param': 0.2,
                'value_loss_coef': 0.5,
                'entropy_coef': 0.01,
                'max_grad_norm': 0.5,
                'ppo_epochs': 4,
                'batch_size': 64,
                'n_steps': 2048,
                'hidden_layers': [256, 256],
                'activation': 'tanh',
                'dropout': 0.0,
                'separate_networks': False,
                'optimizer': 'adam',
                'lr_schedule': 'constant'  # constant, linear, cosine
            }
        
        self.config = config
        
        # Создаем сеть
        self.network = ConfigurablePPONetwork(
            state_size, action_size,
            hidden_layers=config['hidden_layers'],
            activation=config['activation'],
            dropout=config['dropout'],
            separate_networks=config['separate_networks']
        ).to(device)
        
        # Оптимизатор
        if config['optimizer'] == 'adam':
            self.optimizer = optim.Adam(self.network.parameters(), lr=config['learning_rate'])
        elif config['optimizer'] == 'rmsprop':
            self.optimizer = optim.RMSprop(self.network.parameters(), lr=config['learning_rate'])
        elif config['optimizer'] == 'sgd':
            self.optimizer = optim.SGD(self.network.parameters(), lr=config['learning_rate'])
        
        # Буферы для сбора данных
        self.states = []
        self.actions = []
        self.rewards = []
        self.dones = []
        self.values = []
        self.log_probs = []
        
        # Метрики
        self.episode_rewards = []
        self.policy_losses = []
        self.value_losses = []
        self.entropy_losses = []
        
        # Счетчики
        self.step_count = 0
        self.episode_count = 0
        
        print(f"🤖 PPO агент создан:")
        print(f"  - Архитектура: {config['hidden_layers']}")
        print(f"  - Активация: {config['activation']}")
        print(f"  - Отдельные сети: {config['separate_networks']}")
        print(f"  - Оптимизатор: {config['optimizer']}")
        print(f"  - Learning rate: {config['learning_rate']}")
        print(f"  - Параметры сети: {sum(p.numel() for p in self.network.parameters())}")
    
    def get_action(self, state):
        """Получение действия."""
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            action, log_prob, value, entropy = self.network.get_action_and_value(state_tensor)
            
            return action.item(), log_prob.item(), value.item()
    
    def store_transition(self, state, action, reward, done, value, log_prob):
        """Сохранение перехода."""
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)
        self.values.append(value)
        self.log_probs.append(log_prob)
    
    def compute_gae(self, next_value=0):
        """Вычисление Generalized Advantage Estimation."""
        advantages = []
        gae = 0
        
        values = self.values + [next_value]
        
        for step in reversed(range(len(self.rewards))):
            delta = self.rewards[step] + self.config['gamma'] * values[step + 1] * (1 - self.dones[step]) - values[step]
            gae = delta + self.config['gamma'] * self.config['gae_lambda'] * (1 - self.dones[step]) * gae
            advantages.insert(0, gae)
        
        returns = [adv + val for adv, val in zip(advantages, self.values)]
        
        return advantages, returns
    
    def update(self, next_value=0):
        """Обновление сети PPO."""
        if len(self.states) < self.config['n_steps']:
            return
        
        # Вычисляем GAE
        advantages, returns = self.compute_gae(next_value)
        
        # Конвертируем в тензоры
        states = torch.FloatTensor(self.states).to(device)
        actions = torch.LongTensor(self.actions).to(device)
        old_log_probs = torch.FloatTensor(self.log_probs).to(device)
        advantages = torch.FloatTensor(advantages).to(device)
        returns = torch.FloatTensor(returns).to(device)
        
        # Нормализация advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # PPO обновления
        for epoch in range(self.config['ppo_epochs']):
            # Перемешиваем данные
            indices = torch.randperm(len(states))
            
            # Мини-батчи
            for start in range(0, len(states), self.config['batch_size']):
                end = start + self.config['batch_size']
                batch_indices = indices[start:end]
                
                batch_states = states[batch_indices]
                batch_actions = actions[batch_indices]
                batch_old_log_probs = old_log_probs[batch_indices]
                batch_advantages = advantages[batch_indices]
                batch_returns = returns[batch_indices]
                
                # Получаем новые log_probs и values
                new_log_probs, new_values, entropy = self.network.evaluate_actions(batch_states, batch_actions)
                
                # Ratio для PPO
                ratio = torch.exp(new_log_probs - batch_old_log_probs)
                
                # Surrogate loss
                surr1 = ratio * batch_advantages
                surr2 = torch.clamp(ratio, 1 - self.config['clip_param'], 1 + self.config['clip_param']) * batch_advantages
                policy_loss = -torch.min(surr1, surr2).mean()
                
                # Value loss
                value_loss = F.mse_loss(new_values.squeeze(), batch_returns)
                
                # Entropy loss
                entropy_loss = -entropy.mean()
                
                # Общая loss
                total_loss = (policy_loss + 
                             self.config['value_loss_coef'] * value_loss + 
                             self.config['entropy_coef'] * entropy_loss)
                
                # Обновление
                self.optimizer.zero_grad()
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), self.config['max_grad_norm'])
                self.optimizer.step()
                
                # Сохраняем метрики
                self.policy_losses.append(policy_loss.item())
                self.value_losses.append(value_loss.item())
                self.entropy_losses.append(entropy_loss.item())
        
        # Очищаем буферы
        self.clear_buffers()
    
    def clear_buffers(self):
        """Очистка буферов."""
        self.states.clear()
        self.actions.clear()
        self.rewards.clear()
        self.dones.clear()
        self.values.clear()
        self.log_probs.clear()
    
    def save_model(self, filepath):
        """Сохранение модели."""
        torch.save({
            'network_state_dict': self.network.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'config': self.config,
            'step_count': self.step_count,
            'episode_count': self.episode_count
        }, filepath)
        print(f"💾 PPO модель сохранена: {filepath}")
    
    def load_model(self, filepath):
        """Загрузка модели."""
        checkpoint = torch.load(filepath, map_location=device)
        self.network.load_state_dict(checkpoint['network_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.step_count = checkpoint['step_count']
        self.episode_count = checkpoint['episode_count']
        print(f"📂 PPO модель загружена: {filepath}")

# Создаем PPO агента
ppo_config = {
    'learning_rate': 3e-4,
    'gamma': 0.99,
    'gae_lambda': 0.95,
    'clip_param': 0.2,
    'value_loss_coef': 0.5,
    'entropy_coef': 0.01,
    'max_grad_norm': 0.5,
    'ppo_epochs': 4,
    'batch_size': 64,
    'n_steps': 2048,
    'hidden_layers': [256, 256],  # Измените архитектуру здесь
    'activation': 'tanh',  # relu, tanh, leaky_relu, elu
    'dropout': 0.0,
    'separate_networks': False,  # True для отдельных сетей
    'optimizer': 'adam',  # adam, rmsprop, sgd
    'lr_schedule': 'constant'
}

ppo_agent = PPOAgent(mock_env_info.observation_size, mock_env_info.action_size, ppo_config)
print("\n✅ PPO агент готов к обучению!")

### Обучение PPO агента

In [ ]:
def train_ppo_agent(agent, env=None, episodes=1000, max_steps=1000, update_freq=10):
    """
    Обучение PPO агента.
    """
    print(f"🚀 Начинаем обучение PPO на {episodes} эпизодов...")
    
    episode_rewards = []
    episode_lengths = []
    
    # Прогресс бар
    pbar = tqdm(range(episodes), desc="PPO Training")
    
    for episode in pbar:
        # Для демонстрации используем mock-среду
        if env is None:
            # Mock environment для демонстрации
            state = np.random.random(agent.state_size)
            episode_reward = 0
            steps = 0
            
            for step in range(max_steps):
                # Получаем действие
                action, log_prob, value = agent.get_action(state)
                
                # Mock переход
                next_state = np.random.random(agent.state_size)
                reward = np.random.random() - 0.5  # Случайная награда
                done = np.random.random() < 0.01  # 1% шанс завершения
                
                # Сохраняем переход
                agent.store_transition(state, action, reward, done, value, log_prob)
                
                state = next_state
                episode_reward += reward
                steps += 1
                agent.step_count += 1
                
                # Обновление каждые n_steps
                if len(agent.states) >= agent.config['n_steps']:
                    next_value = agent.get_action(state)[2] if not done else 0
                    agent.update(next_value)
                
                if done:
                    break
        else:
            # Реальная Unity среда
            env.reset()
            decision_steps, terminal_steps = env.get_steps(agent.behavior_name)
            
            episode_reward = 0
            steps = 0
            
            while len(decision_steps) > 0 and steps < max_steps:
                # Получаем наблюдения
                observations = decision_steps.obs[0]
                
                # Получаем действия для всех агентов
                actions = []
                log_probs = []
                values = []
                
                for obs in observations:
                    action, log_prob, value = agent.get_action(obs)
                    actions.append(action)
                    log_probs.append(log_prob)
                    values.append(value)
                
                # Отправляем действия в Unity
                action_tuple = ActionTuple(discrete=np.array([actions]))
                env.set_actions(agent.behavior_name, action_tuple)
                env.step()
                
                # Получаем результаты
                decision_steps, terminal_steps = env.get_steps(agent.behavior_name)
                
                # Обработка terminal steps
                for i, agent_id in enumerate(terminal_steps.agent_id):
                    reward = terminal_steps.reward[i]
                    episode_reward += reward
                    
                    # Сохраняем переход
                    agent.store_transition(
                        observations[i], actions[i], reward, True, 
                        values[i], log_probs[i]
                    )
                
                steps += 1
                agent.step_count += 1
                
                # Обновление каждые n_steps
                if len(agent.states) >= agent.config['n_steps']:
                    agent.update()
        
        episode_rewards.append(episode_reward)
        episode_lengths.append(steps)
        agent.episode_count += 1
        
        # Обновление прогресса
        if episode % 10 == 0:
            avg_reward = np.mean(episode_rewards[-10:])
            pbar.set_postfix({
                'Avg Reward': f'{avg_reward:.2f}',
                'Steps': agent.step_count,
                'Buffer': len(agent.states)
            })
        
        # Сохранение модели
        if episode % 100 == 0 and episode > 0:
            agent.save_model(f'ppo_model_episode_{episode}.pth')
    
    return episode_rewards, episode_lengths

# Обучение PPO (демонстрация на mock-среде)
print("⚠️  Демонстрация обучения на mock-среде (замените на реальную Unity среду)")
ppo_rewards, ppo_lengths = train_ppo_agent(ppo_agent, episodes=100)

print(f"\n✅ Обучение PPO завершено!")
print(f"  - Средняя награда: {np.mean(ppo_rewards):.2f}")
print(f"  - Средняя длина эпизода: {np.mean(ppo_lengths):.2f}")
print(f"  - Общее количество шагов: {ppo_agent.step_count}")

## 5. Сравнение результатов <a id="5"></a>

Визуализируем и сравниваем результаты обучения DQN и PPO.

In [ ]:
def plot_training_results(dqn_rewards, ppo_rewards, dqn_lengths, ppo_lengths):
    """
    Построение графиков сравнения результатов обучения.
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Скользящее среднее для сглаживания
    def moving_average(data, window=10):
        return pd.Series(data).rolling(window=window, min_periods=1).mean()
    
    # График 1: Награды по эпизодам
    axes[0, 0].plot(moving_average(dqn_rewards), label='DQN', color='blue', alpha=0.7)
    axes[0, 0].plot(moving_average(ppo_rewards), label='PPO', color='red', alpha=0.7)
    axes[0, 0].set_title('Награды по эпизодам (скользящее среднее)')
    axes[0, 0].set_xlabel('Эпизод')
    axes[0, 0].set_ylabel('Награда')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # График 2: Длина эпизодов
    axes[0, 1].plot(moving_average(dqn_lengths), label='DQN', color='blue', alpha=0.7)
    axes[0, 1].plot(moving_average(ppo_lengths), label='PPO', color='red', alpha=0.7)
    axes[0, 1].set_title('Длина эпизодов (скользящее среднее)')
    axes[0, 1].set_xlabel('Эпизод')
    axes[0, 1].set_ylabel('Шаги')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # График 3: Распределение наград
    axes[1, 0].hist(dqn_rewards, bins=30, alpha=0.7, label='DQN', color='blue')
    axes[1, 0].hist(ppo_rewards, bins=30, alpha=0.7, label='PPO', color='red')
    axes[1, 0].set_title('Распределение наград')
    axes[1, 0].set_xlabel('Награда')
    axes[1, 0].set_ylabel('Частота')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # График 4: Накопленные награды
    cumulative_dqn = np.cumsum(dqn_rewards)
    cumulative_ppo = np.cumsum(ppo_rewards)
    axes[1, 1].plot(cumulative_dqn, label='DQN', color='blue', alpha=0.7)
    axes[1, 1].plot(cumulative_ppo, label='PPO', color='red', alpha=0.7)
    axes[1, 1].set_title('Накопленные награды')
    axes[1, 1].set_xlabel('Эпизод')
    axes[1, 1].set_ylabel('Накопленная награда')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Статистики
    print("📊 Статистики обучения:")
    print(f"\nDQN:")
    print(f"  - Средняя награда: {np.mean(dqn_rewards):.3f} ± {np.std(dqn_rewards):.3f}")
    print(f"  - Максимальная награда: {np.max(dqn_rewards):.3f}")
    print(f"  - Минимальная награда: {np.min(dqn_rewards):.3f}")
    print(f"  - Средняя длина эпизода: {np.mean(dqn_lengths):.1f}")
    
    print(f"\nPPO:")
    print(f"  - Средняя награда: {np.mean(ppo_rewards):.3f} ± {np.std(ppo_rewards):.3f}")
    print(f"  - Максимальная награда: {np.max(ppo_rewards):.3f}")
    print(f"  - Минимальная награда: {np.min(ppo_rewards):.3f}")
    print(f"  - Средняя длина эпизода: {np.mean(ppo_lengths):.1f}")

# Построение графиков
plot_training_results(dqn_rewards, ppo_rewards, dqn_lengths, ppo_lengths)

In [ ]:
def plot_loss_curves(dqn_agent, ppo_agent):
    """
    Построение кривых потерь для обоих алгоритмов.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # DQN потери
    if dqn_agent.losses:
        axes[0].plot(dqn_agent.losses, color='blue', alpha=0.7)
        axes[0].set_title('DQN: Q-Learning Loss')
        axes[0].set_xlabel('Обновление')
        axes[0].set_ylabel('Loss')
        axes[0].grid(True, alpha=0.3)
    
    # PPO потери
    if ppo_agent.policy_losses:
        axes[1].plot(ppo_agent.policy_losses, label='Policy Loss', color='red', alpha=0.7)
        axes[1].plot(ppo_agent.value_losses, label='Value Loss', color='green', alpha=0.7)
        axes[1].plot(ppo_agent.entropy_losses, label='Entropy Loss', color='orange', alpha=0.7)
        axes[1].set_title('PPO: Различные потери')
        axes[1].set_xlabel('Обновление')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
    
    # Epsilon decay для DQN
    if dqn_agent.epsilons:
        axes[2].plot(dqn_agent.epsilons, color='purple', alpha=0.7)
        axes[2].set_title('DQN: Epsilon Decay')
        axes[2].set_xlabel('Обновление')
        axes[2].set_ylabel('Epsilon')
        axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Построение кривых потерь
plot_loss_curves(dqn_agent, ppo_agent)

In [ ]:
def create_performance_report(dqn_agent, ppo_agent, dqn_rewards, ppo_rewards):
    """
    Создание отчета о производительности.
    """
    report = {
        'DQN': {
            'Архитектура': dqn_agent.config['hidden_layers'],
            'Параметры сети': sum(p.numel() for p in dqn_agent.q_network.parameters()),
            'Средняя награда': np.mean(dqn_rewards),
            'Стандартное отклонение': np.std(dqn_rewards),
            'Максимальная награда': np.max(dqn_rewards),
            'Минимальная награда': np.min(dqn_rewards),
            'Финальный epsilon': dqn_agent.epsilon,
            'Размер буфера': len(dqn_agent.replay_buffer),
            'Эпизоды': dqn_agent.episode_count,
            'Шаги': dqn_agent.step_count
        },
        'PPO': {
            'Архитектура': ppo_agent.config['hidden_layers'],
            'Параметры сети': sum(p.numel() for p in ppo_agent.network.parameters()),
            'Средняя награда': np.mean(ppo_rewards),
            'Стандартное отклонение': np.std(ppo_rewards),
            'Максимальная награда': np.max(ppo_rewards),
            'Минимальная награда': np.min(ppo_rewards),
            'Clip параметр': ppo_agent.config['clip_param'],
            'Learning rate': ppo_agent.config['learning_rate'],
            'Эпизоды': ppo_agent.episode_count,
            'Шаги': ppo_agent.step_count
        }
    }
    
    # Создаем DataFrame для красивого отображения
    df = pd.DataFrame(report).T
    
    print("📋 Отчет о производительности:")
    print("=" * 60)
    
    for metric in df.columns:
        print(f"\n{metric}:")
        for algorithm in df.index:
            value = df.loc[algorithm, metric]
            if isinstance(value, float):
                print(f"  {algorithm}: {value:.4f}")
            else:
                print(f"  {algorithm}: {value}")
    
    # Сохраняем отчет
    with open('performance_report.txt', 'w', encoding='utf-8') as f:
        f.write("Отчет о производительности Unity ML-Agents\n")
        f.write("=" * 50 + "\n\n")
        
        for metric in df.columns:
            f.write(f"{metric}:\n")
            for algorithm in df.index:
                value = df.loc[algorithm, metric]
                if isinstance(value, float):
                    f.write(f"  {algorithm}: {value:.4f}\n")
                else:
                    f.write(f"  {algorithm}: {value}\n")
            f.write("\n")
    
    print("\n💾 Отчет сохранен в 'performance_report.txt'")
    
    return report

# Создаем отчет
performance_report = create_performance_report(dqn_agent, ppo_agent, dqn_rewards, ppo_rewards)

## 6. Troubleshooting <a id="6"></a>

Решение распространенных проблем при работе с Unity ML-Agents через внешний API.

### Проблемы с подключением к Unity

**Проблема 1: UnityTimeoutException**
```python
# Решение: Увеличьте таймаут или проверьте порты
env = UnityEnvironment(
    file_name="path/to/your/environment.exe",
    base_port=5005,
    timeout_wait=60  # Увеличьте таймаут
)
```

**Проблема 2: Порт уже занят**
```python
# Решение: Используйте другой порт или найдите свободный
import socket

def find_free_port(start_port=5005):
    for port in range(start_port, start_port + 100):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('', port))
                return port
        except OSError:
            continue
    return None

free_port = find_free_port()
print(f"Свободный порт: {free_port}")
```

**Проблема 3: Exe-файл не найден**
```python
# Решение: Проверьте путь к файлу
import os

def check_unity_exe(path):
    if not os.path.exists(path):
        print(f"❌ Файл не найден: {path}")
        return False
    
    if not path.endswith('.exe'):
        print(f"❌ Не exe-файл: {path}")
        return False
    
    print(f"✅ Файл найден: {path}")
    return True

# Проверка
exe_path = "./YourUnityEnvironment.exe"
check_unity_exe(exe_path)
```

### Проблемы с обучением

**Проблема 1: Медленное обучение**
```python
# Решение: Оптимизация гиперпараметров
optimized_dqn_config = {
    'learning_rate': 0.0005,  # Уменьшите для стабильности
    'batch_size': 128,        # Увеличьте для более стабильного обучения
    'target_update_freq': 500, # Уменьшите для частых обновлений
    'epsilon_decay': 0.999,   # Медленнее уменьшайте exploration
    'gamma': 0.995,          # Больше внимания к будущим наградам
    'hidden_layers': [512, 256, 128]  # Увеличьте размер сети
}

optimized_ppo_config = {
    'learning_rate': 0.0003,
    'n_steps': 4096,         # Больше шагов для сбора данных
    'batch_size': 256,       # Больший размер батча
    'ppo_epochs': 8,         # Больше эпох обучения
    'clip_param': 0.1,       # Меньший clip для стабильности
    'entropy_coef': 0.02     # Больше exploration
}
```

**Проблема 2: Нестабильное обучение**
```python
# Решение: Градиентный клиппинг и нормализация
def stable_update(self, loss):
    self.optimizer.zero_grad()
    loss.backward()
    
    # Градиентный клиппинг
    torch.nn.utils.clip_grad_norm_(self.network.parameters(), max_norm=1.0)
    
    # Проверка на NaN
    for param in self.network.parameters():
        if torch.isnan(param.grad).any():
            print("⚠️  Обнаружен NaN в градиентах!")
            return
    
    self.optimizer.step()
```

**Проблема 3: Переобучение**
```python
# Решение: Regularization и early stopping
class EarlyStopping:
    def __init__(self, patience=20, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        
    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.best_score = val_score
            self.counter = 0
        return False

# Использование
early_stopping = EarlyStopping(patience=50)
```

### Оптимизация производительности

**Многопроцессорное обучение**
```python
# Для параллельного обучения нескольких сред
def create_multiple_environments(base_port=5005, num_envs=4):
    envs = []
    for i in range(num_envs):
        try:
            env = UnityEnvironment(
                file_name="path/to/environment.exe",
                base_port=base_port + i,
                no_graphics=True,  # Отключаем графику для скорости
                worker_id=i
            )
            envs.append(env)
        except Exception as e:
            print(f"Ошибка создания среды {i}: {e}")
    
    return envs
```

**Оптимизация памяти**
```python
# Очистка памяти GPU
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"GPU память очищена. Свободно: {torch.cuda.memory_reserved()} bytes")

# Мониторинг использования памяти
def monitor_memory():
    import psutil
    
    # RAM
    ram = psutil.virtual_memory()
    print(f"RAM: {ram.percent}% используется")
    
    # GPU
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / 1024**3  # GB
        print(f"GPU: {gpu_memory:.2f} GB используется")

# Вызов в процессе обучения
monitor_memory()
```

### Дополнительные утилиты

**Логирование и мониторинг**
```python
import logging
from datetime import datetime

# Настройка логирования
def setup_logging(log_level=logging.INFO):
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(f'training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

logger = setup_logging()
logger.info("Начинаем обучение Unity ML-Agents")
```

**Checkpoint система**
```python
class CheckpointManager:
    def __init__(self, save_dir="checkpoints"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        
    def save_checkpoint(self, agent, episode, metrics):
        checkpoint = {
            'episode': episode,
            'model_state_dict': agent.network.state_dict(),
            'optimizer_state_dict': agent.optimizer.state_dict(),
            'config': agent.config,
            'metrics': metrics,
            'timestamp': datetime.now().isoformat()
        }
        
        filepath = os.path.join(self.save_dir, f'checkpoint_episode_{episode}.pth')
        torch.save(checkpoint, filepath)
        print(f"💾 Checkpoint сохранен: {filepath}")
        
        # Удаляем старые checkpoints (оставляем только последние 5)
        self.cleanup_old_checkpoints()
    
    def cleanup_old_checkpoints(self, keep_last=5):
        checkpoints = [f for f in os.listdir(self.save_dir) if f.startswith('checkpoint_')]
        if len(checkpoints) > keep_last:
            checkpoints.sort()
            for checkpoint in checkpoints[:-keep_last]:
                os.remove(os.path.join(self.save_dir, checkpoint))

# Использование
checkpoint_manager = CheckpointManager()
```

## Заключение

Этот notebook предоставляет полный framework для обучения Unity ML-Agents через внешний API с использованием DQN и PPO алгоритмов. 

### Ключевые особенности:

1. **Полностью настраиваемые архитектуры** - Вы можете легко изменить структуру нейронных сетей
2. **Два алгоритма обучения** - DQN для дискретных действий и PPO для более стабильного обучения
3. **Подключение к Unity** - Работа с exe-файлами Unity среды через API
4. **Визуализация результатов** - Подробные графики и метрики
5. **Troubleshooting** - Решения распространенных проблем

### Следующие шаги:

1. Замените mock-среду на реальную Unity среду
2. Настройте гиперпараметры под вашу конкретную задачу
3. Экспериментируйте с различными архитектурами сетей
4. Используйте TensorBoard для более детального мониторинга
5. Сохраняйте и загружайте лучшие модели для дальнейшего использования

### Полезные ресурсы:

- [Unity ML-Agents Documentation](https://github.com/Unity-Technologies/ml-agents)
- [PyTorch Documentation](https://pytorch.org/docs/stable/index.html)
- [Stable Baselines3 Documentation](https://stable-baselines3.readthedocs.io/)

Удачи в обучении ваших Unity ML-Agents! 🚀